type: 1 조현병(정신분열증) - 자유게시판
      2 조현병(정신분열증) - 상담실
      3 ADHD - 자유게시판
      4 ADHD - 익명게시판

In [2]:
import requests
import sqlite3
import datetime
from bs4 import BeautifulSoup

con = sqlite3.connect("cafe.db")
cur = con.cursor()
cur.execute('select * from data where type is "1"')
schizophrenia_free = cur.fetchall()
cur.execute('select * from data where type is "2"')
schizophrenia_qna = cur.fetchall()
cur.execute('select * from data where type is "3"')
adhd_free = cur.fetchall()
cur.execute('select * from data where type is "4"')
adhd_anonymous = cur.fetchall()
con.commit()
con.close()

In [3]:
con = sqlite3.connect("cafe_data.db")
cur = con.cursor()
cur.execute('select * from data where category is "1"')
sad = cur.fetchall()
con.commit()
con.close()

In [4]:
print("조현병-자유게시판:" + str(len(schizophrenia_free)))
print("조현병-상담실:" + str(len(schizophrenia_qna)))
print("ADHD-자유게시판:" + str(len(adhd_free)))
print("ADHD-익명게시판:" + str(len(adhd_anonymous)))
print("우울증-자유게시판:" + str(len(sad)))

조현병-자유게시판:2360
조현병-상담실:11364
ADHD-자유게시판:1390
ADHD-익명게시판:3727
우울증-자유게시판:9855


## Data 전처리

In [5]:
from soynlp.normalizer import *

temp = schizophrenia_free
for i in range(0,len(temp),1):
    temp[i] = list(temp[i])
    temp[i][2] = only_hangle(temp[i][2])
schizophrenia_free = temp

In [6]:
from soynlp.normalizer import *

temp = schizophrenia_qna
for i in range(0,len(temp),1):
    temp[i] = list(temp[i])
    temp[i][2] = only_hangle(temp[i][2])
schizophrenia_qna = temp

In [7]:
from soynlp.normalizer import *

temp = adhd_free
for i in range(0,len(temp),1):
    temp[i] = list(temp[i])
    temp[i][2] = only_hangle(temp[i][2])
adhd_free = temp

In [8]:
from soynlp.normalizer import *

temp = adhd_anonymous
for i in range(0,len(temp),1):
    temp[i] = list(temp[i])
    temp[i][2] = only_hangle(temp[i][2])
adhd_anonymous = temp

In [9]:
from soynlp.normalizer import *

temp = sad
for i in range(0,len(temp),1):
    temp[i] = list(temp[i])
    temp[i][2] = only_hangle(temp[i][2])
sad = temp

# Schizophrenia data (조현병)

In [11]:
import ngram_model
ngram_schizophrenia_free = []
ngrams = ngram_model.Ngram()
for i in range(0,len(schizophrenia_free),1):
    temp = ngrams.word_ngram(schizophrenia_free[i][2],2,{})
    ngram_schizophrenia_free = ngram_schizophrenia_free + temp
fre_schizophrenia_free = ngrams.make_freqlist(ngram_schizophrenia_free)

In [12]:
import ngram_model
ngram_schizophrenia_qna = []
ngrams = ngram_model.Ngram()
for i in range(0,len(schizophrenia_qna),1):
    temp = ngrams.word_ngram(schizophrenia_qna[i][2],2,{})
    ngram_schizophrenia_qna = ngram_schizophrenia_qna + temp
fre_schizophrenia_qna = ngrams.make_freqlist(ngram_schizophrenia_qna)

# ADHD Data (과잉행동장애)

In [13]:
import ngram_model
ngram_adhd_free = []
ngrams = ngram_model.Ngram()
for i in range(0,len(adhd_free),1):
    temp = ngrams.word_ngram(adhd_free[i][2],2,{})
    ngram_adhd_free = ngram_adhd_free + temp
fre_adhd_free = ngrams.make_freqlist(ngram_adhd_free)

In [14]:
import ngram_model
ngram_adhd_anonymous = []
ngrams = ngram_model.Ngram()
for i in range(0,len(adhd_anonymous),1):
    temp = ngrams.word_ngram(adhd_anonymous[i][2],2,{})
    ngram_adhd_anonymous = ngram_adhd_anonymous + temp
fre_adhd_anonymous = ngrams.make_freqlist(ngram_adhd_anonymous)

# SAD Data (우울증)

In [15]:
import ngram_model
ngram_sad = []
ngrams = ngram_model.Ngram()
for i in range(0,len(sad),1):
    temp = ngrams.word_ngram(sad[i][2],2,{})
    ngram_sad = ngram_sad + temp
fre_sad = ngrams.make_freqlist(ngram_sad)

# Frequency 측정하여 빈도가 잦은 상위 800개의 2-Gram 선정
### 해당 상위 2-Gram 추출

In [16]:
from collections import OrderedDict
from itertools import islice

schizophrenia_free_descending = OrderedDict(sorted(fre_schizophrenia_free.items(), 
                                  key=lambda kv: kv[1], reverse=True))
schizophrenia_qna_descending = OrderedDict(sorted(fre_schizophrenia_qna.items(), 
                                  key=lambda kv: kv[1], reverse=True))
adhd_free_descending = OrderedDict(sorted(fre_adhd_free.items(), 
                                  key=lambda kv: kv[1], reverse=True))
adhd_anonymous_descending = OrderedDict(sorted(fre_adhd_anonymous.items(), 
                                  key=lambda kv: kv[1], reverse=True))
sad_descending = OrderedDict(sorted(fre_sad.items(), 
                                  key=lambda kv: kv[1], reverse=True))

sliced = islice(schizophrenia_free_descending.items(), 800)
sliced_schizophrenia_free_descending = OrderedDict(sliced)
sliced = islice(schizophrenia_qna_descending.items(), 800)
sliced_schizophrenia_qna_descending = OrderedDict(sliced)
sliced = islice(adhd_free_descending.items(), 800)
sliced_adhd_free_descending = OrderedDict(sliced)
sliced = islice(adhd_anonymous_descending.items(), 800)
sliced_adhd_anonymous_descending = OrderedDict(sliced)
sliced = islice(sad_descending.items(), 800)
sliced_sad_descending = OrderedDict(sliced)

In [17]:
print(sliced_schizophrenia_free_descending)

OrderedDict([(('수', '있다'), 1541), (('하다', '하다'), 1364), (('사람', '들'), 1208), (('생각', '하다'), 1157), (('말', '하다'), 1076), (('하다', '것'), 1027), (('못', '하다'), 824), (('하다', '수'), 715), (('하다', '있다'), 713), (('하다', '않다'), 570), (('환', '우'), 551), (('것', '같다'), 540), (('위', '하다'), 539), (('안', '되다'), 493), (('하다', '그'), 462), (('수', '없다'), 458), (('것', '이다'), 457), (('분', '들'), 455), (('되다', '것'), 453), (('일', '하다'), 451), (('입원', '하다'), 440), (('그', '들'), 438), (('감사', '하다'), 434), (('시작', '하다'), 434), (('월', '일'), 407), (('이', '것'), 383), (('하다', '때'), 374), (('하다', '되다'), 370), (('원장', '님'), 362), (('있다', '것'), 357), (('약', '먹다'), 347), (('가족', '들'), 346), (('하다', '이'), 324), (('하다',), 322), (('하다', '사람'), 300), (('우', '들다'), 296), (('있다', '하다'), 292), (('이기다', '내다'), 286), (('복용', '하다'), 278), (('내다', '사람'), 268), (('것', '아니다'), 266), (('하다', '저'), 251), (('인', '하다'), 248), (('사랑', '하다'), 248), (('되다', '않다'), 240), (('하다', '우리'), 240), (('기도', '하다'), 239), (('이야기', '하다'), 239), (('정신분열병'

In [18]:
print(sliced_schizophrenia_qna_descending)

OrderedDict([(('상담', '하다'), 11623), (('상담', '글'), 11608), (('하다', '하다'), 10154), (('복용', '하다'), 9392), (('월', '일'), 8902), (('사람', '들'), 8488), (('진단', '명'), 7890), (('하다', '약'), 7773), (('이전', '상담'), 7770), (('키', '체중'), 7638), (('약', '명'), 7275), (('재발', '유무'), 7156), (('나이', '성별'), 7020), (('체중', '변화'), 6879), (('하다', '분'), 6872), (('명', '용량'), 6753), (('변화', '진단'), 6452), (('년', '월'), 6416), (('용량', '재발'), 6387), (('명', '복용'), 6382), (('유무', '이전'), 6370), (('성별', '키'), 6348), (('분', '나이'), 6346), (('이기다', '내다'), 6333), (('내다', '사람'), 6303), (('정신분열병', '이기다'), 6247), (('정신과', '전문의'), 6173), (('님', '정신과'), 6145), (('전문의', '로서'), 6145), (('로서', '정신분열병'), 6142), (('하다', '상담'), 5949), (('저자', '이다'), 5927), (('하다', '바라다'), 5783), (('수', '있다'), 5337), (('글', '읽다'), 5191), (('체중', '체중'), 5032), (('입원', '하다'), 5028), (('기재', '하다'), 4762), (('하다', '주다'), 4725), (('복사', '금지'), 4715), (('금지', '설정'), 4708), (('설정', '풀'), 4677), (('상담', '반드시'), 4677), (('풀', '허용'), 4675), (('허용', '상담'), 4663), (

In [19]:
print(sliced_adhd_free_descending)

OrderedDict([(('수', '있다'), 2166), (('하다', '것'), 1101), (('하다', '수'), 981), (('하다', '하다'), 974), (('사람', '들'), 797), (('생각', '하다'), 696), (('말', '하다'), 689), (('못', '하다'), 674), (('것', '같다'), 635), (('분', '들'), 497), (('위', '하다'), 495), (('하다', '않다'), 455), (('있다', '것'), 436), (('하다', '있다'), 422), (('하다', '되다'), 386), (('하다', '때'), 370), (('수', '없다'), 365), (('되다', '것'), 335), (('안', '되다'), 321), (('사회', '적'), 318), (('시작', '하다'), 300), (('월', '일'), 295), (('하다', '못'), 289), (('하다', '분'), 285), (('집중', '력'), 283), (('일', '하다'), 275), (('아이', '들'), 258), (('하다', '이'), 257), (('하다', '그'), 252), (('있다', '하다'), 250), (('하다', '사람'), 247), (('도움', '되다'), 236), (('이', '것'), 227), (('것', '아니다'), 207), (('자', '조'), 204), (('조', '모임'), 199), (('다른', '사람'), 195), (('하다',), 189), (('이해', '하다'), 188), (('사용', '하다'), 185), (('인', '하다'), 183), (('되다', '수'), 180), (('수도', '있다'), 178), (('것', '이다'), 174), (('집중', '하다'), 172), (('하다', '보다'), 171), (('하다', '경우'), 169), (('이', '들'), 166), (('원', '하다'), 166

In [20]:
adhd_anonymous_descending = OrderedDict(sorted(fre_adhd_anonymous.items(), 
                                  key=lambda kv: kv[1], reverse=True))
sliced = islice(adhd_anonymous_descending.items(), 800)
sliced_adhd_anonymous_descending = OrderedDict(sliced)
print(sliced_adhd_anonymous_descending)

OrderedDict([(('생각', '하다'), 1265), (('못', '하다'), 1238), (('수', '있다'), 1207), (('하다', '하다'), 1085), (('사람', '들'), 1031), (('안', '되다'), 902), (('말', '하다'), 871), (('하다', '것'), 750), (('약', '먹다'), 718), (('것', '같다'), 716), (('하다', '수'), 567), (('하다', '때'), 563), (('하다', '있다'), 526), (('일', '하다'), 493), (('하다', '않다'), 489), (('이', '것'), 422), (('하다', '되다'), 401), (('시작', '하다'), 389), (('집중', '력'), 372), (('분', '들'), 356), (('남', '들'), 347), (('수', '없다'), 344), (('하다',), 321), (('공부', '하다'), 317), (('복용', '하다'), 306), (('노력', '하다'), 296), (('하다', '못'), 285), (('하다', '사람'), 283), (('하다', '생각'), 279), (('하다', '그'), 266), (('하다', '제'), 256), (('진단', '받다'), 255), (('하다', '저'), 254), (('친구', '들'), 248), (('하다', '이'), 247), (('안', '하다'), 242), (('있다', '하다'), 242), (('있다', '것'), 238), (('위', '하다'), 234), (('집중', '하다'), 231), (('처방', '받다'), 230), (('되다', '것'), 224), (('치료', '받다'), 223), (('이해', '하다'), 222), (('하다', '싶다'), 221), (('어떻다', '하다'), 219), (('것', '아니다'), 216), (('얘기', '하다'), 214), (('하다',

In [21]:
print(sliced_sad_descending)

OrderedDict([(('생각', '하다'), 2569), (('사람', '들'), 1858), (('하다', '하다'), 1813), (('못', '하다'), 1689), (('안', '되다'), 1586), (('약', '먹다'), 1498), (('것', '같다'), 1497), (('말', '하다'), 1454), (('수', '있다'), 1392), (('일', '하다'), 915), (('하다', '있다'), 899), (('시작', '하다'), 822), (('하다', '않다'), 795), (('무기', '력'), 772), (('하다', '것'), 758), (('하다', '수'), 701), (('하다', '되다'), 689), (('하다', '때'), 634), (('이', '것'), 619), (('하다',), 616), (('아', '무것'), 615), (('생각', '들다'), 614), (('하다', '싶다'), 608), (('어떻다', '하다'), 602), (('친구', '들'), 568), (('하다', '사람'), 534), (('안', '하다'), 526), (('하다', '생각'), 523), (('수', '없다'), 522), (('얘기', '하다'), 521), (('하다', '저'), 518), (('분', '들'), 517), (('너무', '힘들다'), 507), (('안', '좋다'), 499), (('안', '먹다'), 487), (('하다', '제'), 482), (('이', '젠'), 481), (('하다', '그렇다'), 471), (('하다', '요'), 461), (('요',), 457), (('우울증', '약'), 450), (('병원', '가다'), 441), (('남', '들'), 440), (('애', '들'), 432), (('다', '들다'), 431), (('하다', '같다'), 428), (('하다', '그'), 426), (('아', '무'), 425), (('노력', '하다')